In [3]:
from RestrictedPython import compile_restricted
from RestrictedPython import safe_globals
from RestrictedPython.Eval import default_guarded_getiter
from RestrictedPython.Guards import guarded_iter_unpack_sequence, guarded_unpack_sequence, safe_builtins


def safe_python_exec(code: str, variables: dict = None) -> dict:
    if variables is None:
        variables = {}

    # Compile code (returns code object or raises SyntaxError)
    try:
        byte_code = compile_restricted(code, filename="<string>", mode="exec")
    except SyntaxError as e:
        return {"error": f"SyntaxError: {e}"}

    # Prepare restricted environment
    restricted_globals = safe_globals.copy()
    restricted_globals.update({
        "__builtins__": safe_builtins,
        "_getiter_": default_guarded_getiter,
        "_iter_unpack_sequence_": guarded_iter_unpack_sequence,
        "_unpack_sequence_": guarded_unpack_sequence,
        "math": __import__("math"),
    })

    restricted_locals = dict(variables)

    try:
        exec(byte_code, restricted_globals, restricted_locals)
    except Exception as e:
        return {"error": str(e)}

    return restricted_locals

In [4]:
code = """
result = math.sin(math.pi / 2) + 5 * 10
"""

output = safe_python_exec(code)

if "error" in output:
    print("Error:", output["error"])
else:
    print("Result:", output.get("result"))


Result: 51.0


In [5]:
output

{'result': 51.0}